In [2]:
import pandas as pd

# HH Data

## Resume

In [3]:
df_resume = pd.read_csv('dst-3.0_16_1_hh_database.csv', delimiter=';')

In [4]:
df_resume

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано
...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Мужчина , 30 лет , родился 17 января 1989",50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","Тверь , готов к переезду (Москва, Химки) , гот...",полная занятость,"полный день, удаленная работа","Опыт работы 7 лет 7 месяцев Финансист, аналит...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,Высшее образование 2015 Московский гуманитарн...,22.04.2019 12:32,Не указано
44740,"Мужчина , 27 лет , родился 5 марта 1992",39000 руб.,"Системный администратор, IT-специалист","Липецк , готов к переезду , готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...","Опыт работы 7 лет Системный администратор, IT...",ИП Пестрецов,Предприниматель,Высшее образование (Бакалавр) 2016 Воронежски...,22.04.2019 13:11,Не указано
44741,"Женщина , 48 лет , родилась 26 декабря 1970",40000 руб.,"Аналитик данных, Математик","Челябинск , готова к переезду , готова к редки...",полная занятость,"полный день, удаленная работа",Опыт работы 21 год 5 месяцев Январь 1998 — по...,"ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,Высшее образование 2000 Южно-Уральский госуда...,09.04.2019 05:07,Не указано
44742,"Мужчина , 24 года , родился 6 октября 1994",20000 руб.,Контент-менеджер,"Тамбов , не готов к переезду , не готов к кома...","частичная занятость, полная занятость",удаленная работа,Опыт работы 3 года 10 месяцев Контент-менедже...,IQ-Maxima,Менеджер проектов,Высшее образование 2015 Тамбовский государств...,26.04.2019 14:25,Имеется собственный автомобиль


In [5]:
df_resume['Ищет работу на должность:'] = df_resume['Ищет работу на должность:'].str.split(', ')

In [6]:
df_resume = df_resume.explode('Ищет работу на должность:')

In [7]:
df_resume['Ищет работу на должность:'] = df_resume['Ищет работу на должность:'].str.lower()

In [8]:
df_resume['Ищет работу на должность:'].value_counts()

системный администратор                                          4114
инженер                                                          1377
аналитик                                                          984
руководитель проекта                                              892
менеджер проектов                                                 842
                                                                 ... 
эксперт по управлению требованиями / руководитель it проектов       1
старший системный аналитик / функциональный архитектор 1с           1
b2c-маркетолог                                                      1
техник аварийно восстановительных работ                             1
head of helpdesk                                                    1
Name: Ищет работу на должность:, Length: 13340, dtype: int64

In [9]:
print(df_resume['Ищет работу на должность:'].unique())

['системный администратор' 'технический писатель' 'оператор' ...
 'тестировщик по работу уже нашёл' 'водитель,продавец-консультант'
 'head of helpdesk']


In [10]:
df_resume = df_resume.reset_index()

In [16]:
df_resume = df_resume.dropna()

In [40]:
def get_experience(exp_str: str) -> str:
    exp_lst = exp_str.split()
    if len(exp_lst) <= 2:
        return 0
    if 'лет' not in ' '.join(exp_lst[:5]) and 'год' not in ' '.join(exp_lst[:5]):
        return int(exp_lst[2]) / 12
    if 'месяц' in ' '.join(exp_lst[:5]):
        return int(exp_lst[2]) + int(exp_lst[4]) / 12
    else:
        return int(exp_lst[2])
        

In [42]:
df_resume['years'] = df_resume['Опыт работы'].apply(lambda x: get_experience(x))

/var/folders/9g/_5ffyp2n731d2wgcprc6zswh0000gn/T/ipykernel_93807/3545115954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resume['years'] = df_resume['Опыт работы'].apply(lambda x: get_experience(x))


In [45]:
df_resume['city'] = df_resume['Город, переезд, командировки'].str.split(',').apply(lambda x: x[0])

/var/folders/9g/_5ffyp2n731d2wgcprc6zswh0000gn/T/ipykernel_93807/93364513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resume['city'] = df_resume['Город, переезд, командировки'].str.split(',').apply(lambda x: x[0])


In [51]:
def get_education(edu_list: list) -> str:
    final = list()
    for word in edu_list:
        if not word.isalpha():
            break
            
        final.append(word)
        
    return ' '.join(final)

In [53]:
df_resume['education'] = df_resume['Образование и ВУЗ'].str.split().apply(get_education)

/var/folders/9g/_5ffyp2n731d2wgcprc6zswh0000gn/T/ipykernel_93807/396589969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resume['education'] = df_resume['Образование и ВУЗ'].str.split().apply(get_education)


In [54]:
df_resume['education'].value_counts()

Высшее образование                 41082
Среднее специальное образование     6693
Неоконченное высшее образование     5314
Среднее образование                  633
Name: education, dtype: int64

In [47]:
df_resume['city'].value_counts()

Москва              19584
Санкт-Петербург      5857
Краснодар            1335
Новосибирск          1175
Казань               1030
                    ...  
Неман                   1
Словения                1
Колтуши                 1
Вьетнам                 1
Темрюк                  1
Name: city, Length: 983, dtype: int64

## Vacancies

In [55]:
df_it_vacancies = pd.read_csv('IT_vacancies_full.csv')

In [57]:
df_it_vacancies.sample(10)

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,Area,Professional roles,Specializations,Profarea names,Published at
9793,46657633,Тинькофф,Бизнес-аналитик (Middle),False,NaN,NaN,От 1 года до 3 лет,Полный день,"['Работа в условиях многозадачности', 'Умение ...","Обязанности Собирать, анализировать требовани...",Москва,['Аналитик'],"['Аналитик', 'Реинжиниринг бизнес процессов', ...","['Информационные технологии, интернет, телеком...",2021-11-29 11:50:33+03:00
16071,49279854,ГК ВессоЛинк,Руководитель проекта,True,80040.0,120060.0,От 3 до 6 лет,Полный день,"['Управление проектами', 'Организаторские навы...",Обязанности: планирование и координация крупн...,Москва,['Руководитель проектов'],['Управление проектами'],"['Информационные технологии, интернет, телеком']",2021-11-28 11:25:12+03:00
188,50108608,TheSoul Publishing (AdMe),Иллюстратор YouTube-проекта,False,NaN,NaN,От 1 года до 3 лет,Удаленная работа,"['Иллюстрирование', 'Художник', 'Креативность'...","Всем привет! Пожалуйста, укажите в сопроводите...",Санкт-Петербург,"['Дизайнер, художник']","['Игровое ПО', 'Программирование, Разработка',...","['Маркетинг, реклама, PR', 'Информационные тех...",2021-12-01 15:17:45+03:00
37067,48542586,Ingram Micro,Cloud Product Owner,False,NaN,NaN,Более 6 лет,Полный день,"['Negotiation skills', 'Teamplayer', 'Leadersh...",Do you crave the creativity and freedom of a s...,Москва,['Менеджер продукта'],"['Программирование, Разработка', 'Продажи', 'У...","['Информационные технологии, интернет, телеком']",2021-11-11 17:24:14+03:00
11235,47862835,"Благо, Компания",Координатор проектов автоматизации,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['MS PowerPoint', 'Работа с большим объемом ин...",В настоящий момент в Компании открыта вакансия...,Санкт-Петербург,['Руководитель проектов'],"['Начальный уровень, Мало опыта', 'Системы упр...","['Административный персонал', 'Информационные ...",2021-11-29 10:48:34+03:00
14902,49954504,Infosuite,Программист 1С,True,87000.0,217500.0,От 3 до 6 лет,Удаленная работа,[],"Компания Infosuite – команда профессионалов, г...",Москва,"['Программист, разработчик']","['Программирование, Разработка', 'Системы упра...","['Информационные технологии, интернет, телеком']",2021-11-28 15:51:11+03:00
19394,49849486,МАЙНТЕК МАШИНЕРИ,Программист 1C,False,NaN,NaN,От 3 до 6 лет,Полный день,"['1С программирование', 'Создание конфигурации...",Нашей компании эксклюзивному дистрибьютору Hit...,Москва,"['Программист, разработчик']","['Программирование, Разработка', 'Системы упра...","['Информационные технологии, интернет, телеком']",2021-11-25 18:01:22+03:00
21930,49918405,VK,Разработчик систем хранения Tarantool,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['Lua', 'Java', 'Linux']",Наша команда занимается разработкой и интеграц...,Москва,"['Программист, разработчик']","['Программирование, Разработка', 'Системы упра...","['Информационные технологии, интернет, телеком']",2021-11-24 15:39:31+03:00
10674,49324375,"Ренессанс cтрахование, Группа",Системный администратор,False,NaN,NaN,Нет опыта,Полный день,"['Linux', 'VMware', 'Zabbix', 'Scrum', 'Agile ...",Делаем мир более безопасным местом с 1997 года...,Санкт-Петербург,['Системный администратор'],"['Передача данных и доступ в интернет', 'Сетев...","['Информационные технологии, интернет, телеком']",2021-11-29 11:16:13+03:00
4211,50043115,АЙТИ.СПЕЙС,С++ программист,False,NaN,NaN,Нет опыта,Удаленная работа,"['Boost', 'C++', 'ООП', 'STL', 'C/C++']",Проект: сложные интерфейсы для электрических а...,Москва,"['Программист, разработчик']","['Программирование, Разработка', 'Компьютерная...","['Информационные технологии, интернет, телеком']",2021-11-29 18:14:05+03:00


In [58]:
df_it_vacancies['Experience'].unique()

array(['От 3 до 6 лет', 'От 1 года до 3 лет', 'Нет опыта', 'Более 6 лет'],
      dtype=object)

In [60]:
df_it_vacancies['Schedule'].unique()

array(['Полный день', 'Удаленная работа', 'Сменный график',
       'Гибкий график', 'Вахтовый метод'], dtype=object)

In [59]:
df_it_vacancies['min_years'] = df_it_vacancies['Experience'].replace({
                                                                        'От 3 до 6 лет': 3, 
                                                                        'От 1 года до 3 лет': 1,
                                                                        'Нет опыта': 0,
                                                                        'Более 6 лет': 6
                                                                    })

In [61]:
df_it_vacancies['Name'] = df_it_vacancies['Name'].str.lower()

In [66]:
names = df_it_vacancies['Name'].value_counts()

names[names <= 1]

middle / senior android developer                                                                1
bi engineer                                                                                      1
full stack разработчик битрикс24                                                                 1
аналитик - руководитель проекта (управление проектной деятельности и аналитической поддержки)    1
сетевой инженер + системный администратор *nix                                                   1
                                                                                                ..
ассистент команды customer success (французский и русский языки)                                 1
программист с++ в отдел вычислительной гидродинамики                                             1
аналитик по подготовке отчетности                                                                1
web-аналитик it-продукта                                                                         1
менеджер п

In [70]:
len(names)

29324

In [73]:
df_it_vacancies['Name'] = df_it_vacancies['Name'].str.split('/').apply(lambda x: x[0])

# Matching

In [74]:
job_resume = df_it_vacancies.merge(df_resume, left_on='Name', right_on='Ищет работу на должность:', how='left')

In [75]:
job_resume.isna().sum()

Ids                                      0
Employer                                 0
Name                                     0
Salary                                   0
From                               2346404
To                                 3157334
Experience                               0
Schedule                                 0
Keys                                     0
Description                              0
Area                                     0
Professional roles                       0
Specializations                          0
Profarea names                           0
Published at                             0
min_years                                0
index                                31756
Пол, возраст                         31756
ЗП                                   31756
Ищет работу на должность:            31756
Город, переезд, командировки         31756
Занятость                            31756
График                               31756
Опыт работы

In [76]:
job_resume = job_resume.dropna(subset=['Ищет работу на должность:'])

In [77]:
job_resume

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,...,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,years,city,education
4,49555567,Pride Games Studio,ux,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['UI', 'UX', 'gamedev', 'game design', 'проект...",Pride Games Studio — это команда единомышленни...,...,"удаленная работа, гибкий график, полный день",Опыт работы 11 лет Март 2018 — Август 2018 6...,Postoport,Веб-дизайнер,Высшее образование (Бакалавр) 2012 Parsons The...,07.05.2019 09:34,Не указано,11.0,Москва,Высшее образование
5,49555567,Pride Games Studio,ux,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['UI', 'UX', 'gamedev', 'game design', 'проект...",Pride Games Studio — это команда единомышленни...,...,удаленная работа,Опыт работы 15 лет 1 месяц Сентябрь 2017 — по...,"Enaza, Playkey",Арт-директор,Среднее специальное образование 2004 ПНИПУ Диз...,13.05.2019 09:58,Имеется собственный автомобиль,15.0,Пермь,Среднее специальное образование
7,45573632,Global Ports Management,специалист технического сопровождения,True,47850.0,56550.0,Нет опыта,Сменный график,"['Грамотная речь', 'Грамотность', 'Работа с бо...",Группа компаний «Глобал Портс» – международная...,...,полный день,Опыт работы 5 лет 4 месяца Февраль 2015 — по ...,"ЦентрИнформ, ФГУП",Специалист технической поддержки пользователей,Высшее образование (Бакалавр) 2014 Санкт-Пете...,13.04.2019 11:15,Не указано,5.0,Санкт-Петербург,Высшее образование
8,45573632,Global Ports Management,специалист технического сопровождения,True,47850.0,56550.0,Нет опыта,Сменный график,"['Грамотная речь', 'Грамотность', 'Работа с бо...",Группа компаний «Глобал Портс» – международная...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 8 лет 6 месяцев Специалист технич...,Атос АйТи Солюшенс энд Сервисез,Инженер технической поддержки,Неоконченное высшее образование 2010 Московск...,22.04.2019 11:06,Не указано,8.0,Москва,Неоконченное высшее образование
10,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,16.0,Советск (Калининградская область),Неоконченное высшее образование
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4581303,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,"полный день, удаленная работа",Опыт работы 11 лет 5 месяцев Январь 2008 — по...,БГАУ,Заведующий отделом САПО,Высшее образование 2018 Башкирский государств...,09.04.2019 08:02,Имеется собственный автомобиль,11.0,Уфа,Высшее образование
4581304,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 3 года 9 месяцев Системный админи...,"Российские железные дороги, ОАО",Инженер-электроник,Высшее образование (Бакалавр) 2015 Петербургс...,22.04.2019 01:02,Не указано,3.0,Санкт-Петербург,Высшее образование
4581305,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,полный день,Опыт работы 4 года 5 месяцев Системный админи...,ГК Дуэт,Системный администратор,Среднее специальное образование 2013 Туйм

In [78]:
job_resume = job_resume.drop_duplicates(subset=['Ids', 'index'])

In [79]:
job_resume

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,...,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,years,city,education
4,49555567,Pride Games Studio,ux,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['UI', 'UX', 'gamedev', 'game design', 'проект...",Pride Games Studio — это команда единомышленни...,...,"удаленная работа, гибкий график, полный день",Опыт работы 11 лет Март 2018 — Август 2018 6...,Postoport,Веб-дизайнер,Высшее образование (Бакалавр) 2012 Parsons The...,07.05.2019 09:34,Не указано,11.0,Москва,Высшее образование
5,49555567,Pride Games Studio,ux,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['UI', 'UX', 'gamedev', 'game design', 'проект...",Pride Games Studio — это команда единомышленни...,...,удаленная работа,Опыт работы 15 лет 1 месяц Сентябрь 2017 — по...,"Enaza, Playkey",Арт-директор,Среднее специальное образование 2004 ПНИПУ Диз...,13.05.2019 09:58,Имеется собственный автомобиль,15.0,Пермь,Среднее специальное образование
7,45573632,Global Ports Management,специалист технического сопровождения,True,47850.0,56550.0,Нет опыта,Сменный график,"['Грамотная речь', 'Грамотность', 'Работа с бо...",Группа компаний «Глобал Портс» – международная...,...,полный день,Опыт работы 5 лет 4 месяца Февраль 2015 — по ...,"ЦентрИнформ, ФГУП",Специалист технической поддержки пользователей,Высшее образование (Бакалавр) 2014 Санкт-Пете...,13.04.2019 11:15,Не указано,5.0,Санкт-Петербург,Высшее образование
8,45573632,Global Ports Management,специалист технического сопровождения,True,47850.0,56550.0,Нет опыта,Сменный график,"['Грамотная речь', 'Грамотность', 'Работа с бо...",Группа компаний «Глобал Портс» – международная...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 8 лет 6 месяцев Специалист технич...,Атос АйТи Солюшенс энд Сервисез,Инженер технической поддержки,Неоконченное высшее образование 2010 Московск...,22.04.2019 11:06,Не указано,8.0,Москва,Неоконченное высшее образование
10,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,16.0,Советск (Калининградская область),Неоконченное высшее образование
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4581303,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,"полный день, удаленная работа",Опыт работы 11 лет 5 месяцев Январь 2008 — по...,БГАУ,Заведующий отделом САПО,Высшее образование 2018 Башкирский государств...,09.04.2019 08:02,Имеется собственный автомобиль,11.0,Уфа,Высшее образование
4581304,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 3 года 9 месяцев Системный админи...,"Российские железные дороги, ОАО",Инженер-электроник,Высшее образование (Бакалавр) 2015 Петербургс...,22.04.2019 01:02,Не указано,3.0,Санкт-Петербург,Высшее образование
4581305,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,полный день,Опыт работы 4 года 5 месяцев Системный админи...,ГК Дуэт,Системный администратор,Среднее специальное образование 2013 Туйм

In [80]:
sum(job_resume['Ids'].value_counts() > 10)

10202

In [81]:
job_resume['Name'].value_counts()

системный администратор             2573208
аналитик                             281834
специалист технической поддержки     196000
системный аналитик                   104346
руководитель проектов                101721
                                     ...   
менеджер проектов (e-commerce)            1
технический аналитик                      1
программист (junior+                      1
архитектор siebel crm                     1
game designer (balance)                   1
Name: Name, Length: 1826, dtype: int64

In [82]:
counts = job_resume['Ids'].value_counts()

In [83]:
counts = counts[counts >= 10]

to_keep = counts.index

job_resume = job_resume[job_resume['Ids'].isin(to_keep)]

In [90]:
job_resume.columns

Index(['Ids', 'Employer', 'Name', 'Salary', 'From', 'To', 'Experience',
       'Schedule', 'Keys', 'Description', 'Area', 'Professional roles',
       'Specializations', 'Profarea names', 'Published at', 'min_years',
       'index', 'Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ', 'Обновление резюме', 'Авто', 'years', 'city',
       'education'],
      dtype='object')

In [85]:
job_resume['Ids'].value_counts()

48957743    4104
49824168    4104
49682004    4104
49683503    4104
49954078    4104
            ... 
47988692      10
47041312      10
49126035      10
47897063      10
49891563      10
Name: Ids, Length: 10550, dtype: int64

In [89]:
job_resume = job_resume[job_resume['years'] >= job_resume['min_years']]

In [94]:
job_resume = job_resume[(job_resume['city']==job_resume['Area'])|(job_resume['Schedule']=='Удаленная работа')]

In [98]:
job_resume

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,...,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,years,city,education
10,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,16.0,Советск (Калининградская область),Неоконченное высшее образование
11,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,полный день,Опыт работы 9 лет 4 месяца Системный админист...,"ООО ""Дилижанс""",Системный администратор,"Высшее образование 2011 МГУПИ ИТ/программист, ...",22.04.2019 13:51,Не указано,9.0,Санкт-Петербург,Высшее образование
12,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 3 года 10 месяцев Системный админ...,МАОУ СОШ №3,лаборант ИВТ,Неоконченное высшее образование 2020 НТСПИ Ест...,22.04.2019 06:05,Не указано,3.0,Нижний Тагил,Неоконченное высшее образование
13,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,полный день,Опыт работы 16 лет 2 месяца Сентябрь 2012 — п...,Государственная дума РФ,Помощник депутата,Высшее образование (Бакалавр) 2017 Московский ...,20.04.2019 16:15,Имеется собственный автомобиль,16.0,Москва,Высшее образование
14,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, удаленная работа",Опыт работы 17 лет 3 месяца Декабрь 2017 — по...,"ООО ""Техкомплект""",Системный администратор,Высшее образование 1999 Красноярский государс...,10.04.2019 15:37,Имеется собственный автомобиль,17.0,Красноярск,Высшее образование
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576806,49110128,Exness,qa automation engineer,True,300000.0,NaN,От 3 до 6 лет,Удаленная работа,"['Python', 'Английский язык', 'JavaScript']","Exness Group, fast moving organization that sp...",...,"гибкий график, полный день",Опыт работы 8 лет 3 месяца QA Automation Engi...,EPAM Systems Inc (Беларусь),Junior Software Test Automation Engineer,Высшее образование 2009 Белорусский националь...,26.04.2019 15:16,Имеется собственный автомобиль,8.0,Минск,Высшее образование
4576807,49110128,Exness,qa automation engineer,True,300000.0,NaN,От 3 до 6 лет,Удаленная работа,"['Python', 'Английский язык', 'JavaScript']","Exness Group, fast moving organization that sp...",...,полный день,Опыт работы 5 лет 10 месяцев Октябрь 2018 — п...,"ООО ""Тета Дата Солюшенс""",Инженер по тестированию ПО,Высшее образование 2012 Ивановская государств...,07.05.2019 12:11,Не указано,5.0,Иваново (Ивановская область),Высшее образование
4576808,49110128,Exness,qa automation engineer,True,300000.0,NaN,От 3 до 6 лет,Удаленная работа,"['Python', 'Английский язык', 'JavaScript']","Exness Group, fast moving organization that sp...",...,полный день,Опыт работы 6 лет 1 месяц QA Automation Engin...,"ЦУМ, ТД",QA Automation Engineer,Высшее образование 2010 Московский автомобиль...,25.04.2019 20:51,Не указано,6.0,Москва,Высшее образование
4576809,49110128,Exness,qa automation engineer,True,300000.0,Na

In [99]:
from sklearn.model_selection import train_test_split

SEED = 3456

In [102]:
train, test_val = train_test_split(job_resume, test_size=0.2, random_state=SEED)

In [103]:
val, test = train_test_split(test_val, test_size=0.4, random_state=SEED)